In [4]:
import pandas as pd
electoral_result = pd.read_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/1976-2020-president.csv')

electoral_result = electoral_result.drop(columns=['state_po', 'state_fips', 'state_cen', 'state_ic', 'writein', 'notes', 'version'])

electoral_result = electoral_result[electoral_result['year'] >= 2008]

electoral_result.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/2008-2020-president.csv')
print(electoral_result.head())


      year    state        office         candidate party_detailed  \
2728  2008  ALABAMA  US PRESIDENT      MCCAIN, JOHN     REPUBLICAN   
2729  2008  ALABAMA  US PRESIDENT  OBAMA, BARACK H.       DEMOCRAT   
2730  2008  ALABAMA  US PRESIDENT               NaN    INDEPENDENT   
2731  2008  ALABAMA  US PRESIDENT               NaN            NaN   
2732  2008   ALASKA  US PRESIDENT      MCCAIN, JOHN     REPUBLICAN   

      candidatevotes  totalvotes party_simplified  
2728         1266546     2099819       REPUBLICAN  
2729          813479     2099819         DEMOCRAT  
2730           16089     2099819            OTHER  
2731            3705     2099819            OTHER  
2732          193841      326197       REPUBLICAN  


In [5]:
winning_percentage = round((electoral_result['candidatevotes'] / electoral_result['totalvotes']) * 100, 2)
electoral_result['winning_percentage'] = winning_percentage


In [6]:
# Drop only the columns that exist in the DataFrame
columns_to_drop = ['candidatevotes', 'totalvotes', 'party_simplified']
existing_columns_to_drop = [col for col in columns_to_drop if col in electoral_result.columns]
electoral_result = electoral_result.drop(columns=existing_columns_to_drop)

filtered_electoral_result = electoral_result[electoral_result['party_detailed'].isin(['REPUBLICAN', 'DEMOCRAT'])]


df = pd.DataFrame(filtered_electoral_result)

# Reshape the DataFrame
merged = df.groupby(['year', 'state', 'office']).apply(lambda group: pd.Series({
    'Winning Candidate': group.loc[group['winning_percentage'].idxmax(), 'candidate'],
    'Winning Party': group.loc[group['winning_percentage'].idxmax(), 'party_detailed'],
    'Winning Percentage': group['winning_percentage'].max(),
    'Losing Candidate': group.loc[group['winning_percentage'].idxmin(), 'candidate'],
    'Losing Party': group.loc[group['winning_percentage'].idxmin(), 'party_detailed'],
    'Losing Percentage': group['winning_percentage'].min()
})).reset_index()


# Rename several rows to fit the format of the other datasets
merged = merged.rename(columns={'state': 'State', 'office': 'Office', 'year': 'Year'})
# Display the merged DataFrame
print(merged)
merged.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_electoral_data.csv', index=False)

     Year          State        Office    Winning Candidate Winning Party  \
0    2008        ALABAMA  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
1    2008         ALASKA  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
2    2008        ARIZONA  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
3    2008       ARKANSAS  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
4    2008     CALIFORNIA  US PRESIDENT     OBAMA, BARACK H.      DEMOCRAT   
..    ...            ...           ...                  ...           ...   
199  2020       VIRGINIA  US PRESIDENT  BIDEN, JOSEPH R. JR      DEMOCRAT   
200  2020     WASHINGTON  US PRESIDENT  BIDEN, JOSEPH R. JR      DEMOCRAT   
201  2020  WEST VIRGINIA  US PRESIDENT     TRUMP, DONALD J.    REPUBLICAN   
202  2020      WISCONSIN  US PRESIDENT  BIDEN, JOSEPH R. JR      DEMOCRAT   
203  2020        WYOMING  US PRESIDENT     TRUMP, DONALD J.    REPUBLICAN   

     Winning Percentage     Losing Candidate Losing Party  Losing Percentag

/tmp/ipykernel_6489/4098722836.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged = df.groupby(['year', 'state', 'office']).apply(lambda group: pd.Series({
